https://stevesie.com/apps/airbnb-api/listings

In [1]:
import requests
import json
import time

In [2]:
url = "https://api.airbnb.com/v2/explore_tabs"
params = {
    'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
    '_format':'for_explore_search_web',
    
    'currency':'KRW',
    'location':'서울',
    'tab_id':'home_tab',
    'items_per_grid':'306', # 최대 306 -> 수정할방법찾아야
#     'section_offset':'1',
#     'supports_for_you_v3':'true',
#     'timezone_offset':'300',
    'version':'1.3.9'
    }

In [31]:
resp = requests.get(url,params=params)

JSON 개념 정리

https://github.com/chankoo/python-for-data/blob/master/3.2%20JSON%2C%20XML.ipynb

In [32]:
res_jsn = resp.json() # .json() 메소드: response의 형태가 json 일경우 이 response를 파이썬 dict로 바로 매핑해준다

In [33]:
# with open('home_jsn_306.json', 'w') as fp: # 파일로 저장하자 -> review 크롤러에서 가져다 쓰기 위해서이다
#     json.dump(res_jsn,fp) # 파이썬 dict를 다시 json으로 변환하려면 json 모듈의 dump를 이용

__데이터 구조 개괄__
- res_jsn: {explore_tabs, metadata}
    -  __explore_tabs[0]__: { 'tab_id', 'tab_name', 'pagination_metadata', 'sections', 'experiments_metadata', 'home_tab_metadata'}
        - tab_id: home_tab
        - tab_name: homes
        - pagination_metadata: {'has_next_page', 'items_offset', 'section_offset', 'search_session_id'}
        - __sections: [0:not paginated, 1: paginated]__
            - sections[0]: ...'messages': [{'icon': 'icon-p2-aggregate-reviews',
                                           'style': 'MESSAGE_WITH_ICON',
                                       'title': 'Over 540,000 guest reviews for Seoul homes, with an average of 4.7 out of 5 stars.'}]...
                                       
            - __sections__[1]:{'backend_search_id', 'display_type', 'experiments_metadata', 'result_type', 'search_session_id', 'section_id', 'section_type_uid', 'title', 'is_paginated', 'bankai_section_id', 'refinements', __'listings'__, 'review_items', 'breadcrumbs', 'section_metadata', 'nearby_locations', 'localized_listing_count'}
                - ...
                - __listings: [#items_per_grid \* { {listing}, {pricing_quote}, {verified}, {verified_card} }]__
                    - __listing : 해당 숙소에 대한 50가지 features__
                - ...
        - experiments_metadata
        - home_tab_metadata: {'urgency_commitment', 'golden_ticket_urgency_commitment', 'listing_cards_urgency_commitment_metadata', 'listing_cards_price_line_urgency_commitment_metadata', 'messages', 'facets', 'overrides', 'search_feed_items', __'remarketing_ids'__, 'location', 'breadcrumbs', 'listings_count', 'search', 'geography', 'price_histogram', 'filters'}
            - __remarketing_ids__ : listing_id 담은 리스트 -> 숙소 id 바로뽑을 수 있다
  
    
    
    - metadata

In [5]:
res_jsn.keys() # 가장 큰 구조, 'explore_tabs'과 'metadata'로 나뉨

dict_keys(['explore_tabs', 'metadata'])

In [6]:
res_jsn['explore_tabs'][0].keys() # explore_tabs을 살펴보자

dict_keys(['tab_id', 'tab_name', 'pagination_metadata', 'sections', 'experiments_metadata', 'home_tab_metadata'])

In [7]:
res_jsn['explore_tabs'][0]['tab_id']
res_jsn['explore_tabs'][0]['tab_name'] # 웹에서 보고있는 탭이 homes 이다

'homes'

> home 탭으로 추정되는 화면
>![2018-11-04 13-38-47](https://user-images.githubusercontent.com/38183218/47960055-11210580-e037-11e8-865d-8f3fda99ed7b.png)

계속 살펴보자

In [8]:
# pagination 즉, 페이지가 넘어가는 것에 대한 메타데이터가 담겨있음
res_jsn['explore_tabs'][0]['pagination_metadata'] # items_per_grid 300개 이상 설정시 {'has_next_page': False} 가 떠버림

{'has_next_page': True,
 'items_offset': 100,
 'section_offset': 5,
 'search_session_id': '3cb445a2-32c0-420a-91df-b38e1c87d51e'}

In [9]:
# 본격적인 컨텐츠를 구성하는 sections[1] 의 구조
res_jsn['explore_tabs'][0]['sections'][1].keys()

dict_keys(['backend_search_id', 'display_type', 'experiments_metadata', 'result_type', 'search_session_id', 'section_id', 'section_type_uid', 'title', 'is_paginated', 'bankai_section_id', 'refinements', 'listings', 'review_items', 'breadcrumbs', 'section_metadata', 'nearby_locations', 'localized_listing_count'])

In [10]:
# listings에 숙소가 리스팅되어있다
res_jsn['explore_tabs'][0]['sections'][1]['listings'][1].keys()

dict_keys(['listing', 'pricing_quote', 'verified', 'verified_card'])

위 4가지 key중 listing에 대부분의 feature가 들어있음 (50개)

__listing dict의 key 설명__

- badges: ?
- bathroom_label, bathrooms: 화장실 수
- bed_label, bedroom_label, bedrooms: 침대 수, 침실 형태, 침실 수
- city: 표기 혼재 ex) '서울특별시', '마포구', 'Mapo-gu'
- guest_label: 수용인원?
- host_languages: 호스트 가능 언어
- host_thumbnail_url_small, host_thumbnail_url: 호스트 썸네일
- __id : 숙소 id__ 인듯?
- is_business_travel_ready: 비즈니스 트래블위한 숙소인지
- is_fully_refundable: 환불가능여부
- is_host_highly_rated: 호스트 신뢰가능 여부

... : 기타 features 생략

- kicker_content: 숙소형태 요약?
- lat, lng: 위도 경도
- localized_city: 군,구 수준 주소?
- localized_neighborhood: 지하철역 수준 주소?
- name: 숙소이름
- neighborhood: 근처 뭐있는지?
- person_capacity: 수용가능인원 (guest_label과 차이는뭐지??)
- picture_count,picture_url,picture_urls,picture: 숙소 사진 관련
- preview_amenities: 프리뷰에 보여지는 amenities들인듯 ex) 와이파이, 주방
- property_type_id: room_and_property_type 관련 id인듯
- __reviews_count: 리뷰수__
- room_type_category, room_type: 숙소형태 ex)Entire home/apt

... 기타

- star_rating: 숙소 별점
- tier_id: ?
- __user: 호스트 관련 정보(id, 썸네일..)__
- public_address: 군,구 수준까지 주소
- seo_reviews: ?
- amenity_ids, preview_amenity_names: 보유한 amenity들??
- reviews: ??


In [11]:
# listing의 키밸류 쌍을 뽑아보자
for k,v in res_jsn['explore_tabs'][0]['sections'][1]['listings'][10]['listing'].items():
     print("|| %30s || : %s" % (k,v)) # (|| key || : value) 형태로 출력


||                         badges || : []
||                 bathroom_label || : 1 bath
||                      bathrooms || : 1.0
||                      bed_label || : 2 beds
||                  bedroom_label || : 2 bedrooms
||                       bedrooms || : 2
||                           beds || : 2
||                           city || : Yongsan-gu
||                    guest_label || : 4 guests
||                 host_languages || : ['en', 'ko']
||       host_thumbnail_url_small || : https://a0.muscache.com/im/pictures/970eb53a-a8bd-4a04-a6dc-861846582771.jpg?aki_policy=profile_small
||             host_thumbnail_url || : https://a0.muscache.com/im/pictures/970eb53a-a8bd-4a04-a6dc-861846582771.jpg?aki_policy=profile_x_medium
||                             id || : 16683321
||       is_business_travel_ready || : False
||            is_fully_refundable || : True
||           is_host_highly_rated || : True
||                 is_new_listing || : False
||                  is_rebooka

In [12]:
res_jsn['explore_tabs'][0]['experiments_metadata'] # 뭐없다

[]

home_tab에 대한 메타데이터도 뽑아보자

결과적으로 여기의 remarketing_ids 에서 숙소 id를 바로 뽑는다

In [13]:
for k,v in res_jsn['explore_tabs'][0]['home_tab_metadata'].items():
    if k == 'filters': continue # filters 너무 많아 출력 안하겠다
    print("|| %30s || : %s" % (k,v)) # (|| key || : value) 형태로 출력

||             urgency_commitment || : {'message_type': 'empty_message', 'message': {}}
|| golden_ticket_urgency_commitment || : {'message_type': 'empty_message', 'message': {}}
|| listing_cards_urgency_commitment_metadata || : []
|| listing_cards_price_line_urgency_commitment_metadata || : []
||                       messages || : {}
||                         facets || : {}
||                      overrides || : {}
||              search_feed_items || : []
||                remarketing_ids || : [8213750, 11444903, 18956008, 12778317, 22320269, 4619104, 19777144, 17560209, 16683321, 16965764, 22133272, 18739581, 18967029, 20719576, 19634147, 20129439, 1753246, 17578320, 10044927, 21007367, 12857880, 14440359, 20763289, 15596154, 18385571, 4258317, 14069246, 19378056, 17481718, 15649684, 9710807, 8996724, 22056285, 20292369, 19576908, 18473432, 1077893, 16013768, 17660903, 14885947, 11802302, 5714483, 19433253, 10127540, 9715695, 7128212, 1634115, 982772, 24129062, 14482939, 20546115, 

In [14]:
# res_jsn['explore_tabs'][0]['home_tab_metadata']['filters'] # 필터 정보 들여다볼 필요있으면 켜보자

In [15]:
res_jsn['metadata'] # 마지막으로 res_json 전체에 대한 metadata이다 어떻게 고쳐야할지...

{'current_tab_id': 'home_tab',
 'request_uuid': '44734ba5-83ed-4b45-8575-dca541f1e5ee',
 'search_id': '44734ba5-83ed-4b45-8575-dca541f1e5ee',
 'query': '서울',
 'query_text': '서울특별시 · Homes',
 'current_refinement_paths': ['/homes'],
 'federated_search_id': '7ac2d6a3-e859-4cf1-aa51-1fe67d422f33',
 'federated_search_session_id': '1952caa0-a801-44bb-a9e7-cb7aabcaff7c',
 'suggestions': [{'title': 'All',
   'type': 'for_you',
   'search_params': {'params': [],
    'query': '서울',
    'refinement_paths': ['/for_you'],
    'refinement_path': '/for_you',
    'tab_id': 'all_tab'},
   'is_selected': False},
  {'title': 'Homes',
   'type': 'homes',
   'search_params': {'params': [],
    'query': '서울',
    'refinement_paths': ['/homes'],
    'refinement_path': '/homes',
    'tab_id': 'home_tab'},
   'is_selected': True},
  {'title': 'Experiences',
   'type': 'experiences',
   'search_params': {'params': [],
    'query': '서울',
    'refinement_paths': ['/experiences'],
    'refinement_path': '/experien

In [43]:
url = "https://api.airbnb.com/v2/explore_tabs"
params = {
    'key':'d306zoyjsyarp7ifhu67rjxn52tv0t20',
    '_format':'for_explore_search_web',
    
    'currency':'KRW',
    'location':'서울',
    'tab_id':'home_tab',
    'items_per_grid':'500', # 최대 306 -> 수정할방법찾아야
#     'section_offset':'1',
#     'supports_for_you_v3':'true',
    'timezone_offset':'300',
    'version':'1.3.9',
    'price_min':'0',
    'price_max':'99',
    }
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}


In [44]:
%%time
home = {}
while int(params['price_max']) < 1000000:
    print(params['price_min'],params['price_max'])
    
    retry = 3 # request 예외발생시 재시도 횟수
    while retry !=0:
        try:
            resp = requests.get(url,params=params)
            res_jsn = resp.json()
            home_count = len(res_jsn['explore_tabs'][0]['sections'][1]['listings'])
            break

        except Exception as e:
            retry -= 1
            print('ERR: btw {} and {}'.format(params['price_min'],params['price_max']),end='|')
            print(e)
            time.sleep(1)
            continue
    
    if retry ==0:
        print('해당 데이터 못가져옴')
        continue
    if home_count > 0:
        print('해당가격대 숙소 count: ',home_count)
        for el in res_jsn['explore_tabs'][0]['sections'][1]['listings']:
            tmp_dict = el['listing']
            tmp_id = str(tmp_dict['id'])
#             if home.get(tmp_id) is not None: print('home_id {} is overlapped, be updated'.format(tmp_id))
            del tmp_dict['id']
            home[tmp_id] = tmp_dict
    
    if int(params['price_max']) < 100000: # 십만원 기준으로 그미만은 100원단위, 이상은 1000원 단위로 request
        params['price_min'] = str(int(params['price_min']) + 100) # 가격필터 변경(100원단위)
        params['price_max'] = str(int(params['price_min']) + 99)
    else:
        params['price_min'] = str(int(params['price_min']) + 1000) # 가격필터 변경(1000원단위)
        params['price_max'] = str(int(params['price_min']) + 999)

0 99
100 199
ERR: btw 100 and 199|'explore_tabs'
200 299
300 399
ERR: btw 300 and 399|Expecting value: line 1 column 1 (char 0)
400 499
500 599
600 699
700 799
800 899
900 999
1000 1099
1100 1199
1200 1299
1300 1399
ERR: btw 1300 and 1399|'explore_tabs'
1400 1499
1500 1599
1600 1699
ERR: btw 1600 and 1699|Expecting value: line 1 column 1 (char 0)
1700 1799
1800 1899
ERR: btw 1800 and 1899|Expecting value: line 1 column 1 (char 0)
ERR: btw 1800 and 1899|Expecting value: line 1 column 1 (char 0)
1900 1999
ERR: btw 1900 and 1999|Expecting value: line 1 column 1 (char 0)
2000 2099
ERR: btw 2000 and 2099|'explore_tabs'
2100 2199
2200 2299
ERR: btw 2200 and 2299|Expecting value: line 1 column 1 (char 0)
2300 2399
ERR: btw 2300 and 2399|'explore_tabs'
2400 2499
2500 2599
2600 2699
2700 2799
ERR: btw 2700 and 2799|'explore_tabs'
2800 2899
2900 2999
3000 3099
3100 3199
ERR: btw 3100 and 3199|'explore_tabs'
3200 3299
3300 3399
3400 3499
ERR: btw 3400 and 3499|'explore_tabs'
3500 3599
ERR: btw 35

ERR: btw 20200 and 20299|'explore_tabs'
해당가격대 숙소 count:  306
20300 20399
해당가격대 숙소 count:  306
20400 20499
해당가격대 숙소 count:  306
20500 20599
해당가격대 숙소 count:  306
20600 20699
해당가격대 숙소 count:  306
20700 20799
해당가격대 숙소 count:  306
20800 20899
해당가격대 숙소 count:  121
20900 20999
해당가격대 숙소 count:  121
21000 21099
해당가격대 숙소 count:  121
21100 21199
해당가격대 숙소 count:  121
21200 21299
해당가격대 숙소 count:  121
21300 21399
ERR: btw 21300 and 21399|'explore_tabs'
해당가격대 숙소 count:  121
21400 21499
해당가격대 숙소 count:  121
21500 21599
해당가격대 숙소 count:  121
21600 21699
해당가격대 숙소 count:  121
21700 21799
해당가격대 숙소 count:  121
21800 21899
해당가격대 숙소 count:  306
21900 21999
ERR: btw 21900 and 21999|'explore_tabs'
해당가격대 숙소 count:  248
22000 22099
해당가격대 숙소 count:  248
22100 22199
해당가격대 숙소 count:  248
22200 22299
해당가격대 숙소 count:  248
22300 22399
해당가격대 숙소 count:  248
22400 22499
해당가격대 숙소 count:  248
22500 22599
해당가격대 숙소 count:  248
22600 22699
해당가격대 숙소 count:  248
22700 22799
해당가격대 숙소 count:  248
22800 22899
해당가격대 숙소 count:  248
2

해당가격대 숙소 count:  306
44400 44499
해당가격대 숙소 count:  306
44500 44599
해당가격대 숙소 count:  306
44600 44699
해당가격대 숙소 count:  306
44700 44799
해당가격대 숙소 count:  306
44800 44899
해당가격대 숙소 count:  306
44900 44999
해당가격대 숙소 count:  306
45000 45099
해당가격대 숙소 count:  306
45100 45199
해당가격대 숙소 count:  306
45200 45299
해당가격대 숙소 count:  306
45300 45399
해당가격대 숙소 count:  306
45400 45499
해당가격대 숙소 count:  306
45500 45599
해당가격대 숙소 count:  104
45600 45699
해당가격대 숙소 count:  104
45700 45799
해당가격대 숙소 count:  104
45800 45899
해당가격대 숙소 count:  104
45900 45999
해당가격대 숙소 count:  104
46000 46099
해당가격대 숙소 count:  104
46100 46199
해당가격대 숙소 count:  104
46200 46299
해당가격대 숙소 count:  104
46300 46399
해당가격대 숙소 count:  104
46400 46499
해당가격대 숙소 count:  104
46500 46599
해당가격대 숙소 count:  254
46600 46699
해당가격대 숙소 count:  150
46700 46799
해당가격대 숙소 count:  150
46800 46899
해당가격대 숙소 count:  150
46900 46999
해당가격대 숙소 count:  150
47000 47099
해당가격대 숙소 count:  150
47100 47199
해당가격대 숙소 count:  150
47200 47299
해당가격대 숙소 count:  150
47300 47399
해당가격대 숙소 c

해당가격대 숙소 count:  209
63400 63499
해당가격대 숙소 count:  89
63500 63599
해당가격대 숙소 count:  89
63600 63699
해당가격대 숙소 count:  89
63700 63799
해당가격대 숙소 count:  89
63800 63899
ERR: btw 63800 and 63899|'explore_tabs'
해당가격대 숙소 count:  89
63900 63999
ERR: btw 63900 and 63999|'explore_tabs'
해당가격대 숙소 count:  89
64000 64099
해당가격대 숙소 count:  89
64100 64199
해당가격대 숙소 count:  89
64200 64299
해당가격대 숙소 count:  89
64300 64399
해당가격대 숙소 count:  89
64400 64499
해당가격대 숙소 count:  89
64500 64599
해당가격대 숙소 count:  306
64600 64699
해당가격대 숙소 count:  304
64700 64799
해당가격대 숙소 count:  304
64800 64899
ERR: btw 64800 and 64899|'explore_tabs'
해당가격대 숙소 count:  304
64900 64999
해당가격대 숙소 count:  304
65000 65099
해당가격대 숙소 count:  304
65100 65199
해당가격대 숙소 count:  304
65200 65299
ERR: btw 65200 and 65299|'explore_tabs'
해당가격대 숙소 count:  304
65300 65399
해당가격대 숙소 count:  304
65400 65499
해당가격대 숙소 count:  304
65500 65599
ERR: btw 65500 and 65599|'explore_tabs'
해당가격대 숙소 count:  304
65600 65699
ERR: btw 65600 and 65699|'explore_tabs'
ERR: btw 656

ERR: btw 78600 and 78699|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 count:  200
78700 78799
ERR: btw 78700 and 78799|Expecting value: line 1 column 1 (char 0)
ERR: btw 78700 and 78799|'explore_tabs'
ERR: btw 78700 and 78799|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
78700 78799
ERR: btw 78700 and 78799|'explore_tabs'
ERR: btw 78700 and 78799|'explore_tabs'
해당가격대 숙소 count:  200
78800 78899
해당가격대 숙소 count:  200
78900 78999
해당가격대 숙소 count:  200
79000 79099
ERR: btw 79000 and 79099|Expecting value: line 1 column 1 (char 0)
ERR: btw 79000 and 79099|'explore_tabs'
해당가격대 숙소 count:  200
79100 79199
해당가격대 숙소 count:  306
79200 79299
해당가격대 숙소 count:  289
79300 79399
해당가격대 숙소 count:  289
79400 79499
ERR: btw 79400 and 79499|'explore_tabs'
ERR: btw 79400 and 79499|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 count:  289
79500 79599
ERR: btw 79500 and 79599|'explore_tabs'
해당가격대 숙소 count:  289
79600 79699
해당가격대 숙소 count:  289
79700 79799
해당가격대 숙소 count:  289
79800 79899
해당가격대

...

해당가격대 숙소 count:  22
144000 144999
해당가격대 숙소 count:  28
145000 145999
ERR: btw 145000 and 145999|'explore_tabs'
ERR: btw 145000 and 145999|'explore_tabs'
해당가격대 숙소 count:  32
146000 146999
해당가격대 숙소 count:  22
147000 147999
해당가격대 숙소 count:  6
148000 148999
해당가격대 숙소 count:  18
149000 149999
해당가격대 숙소 count:  126
150000 150999
해당가격대 숙소 count:  123
151000 151999
해당가격대 숙소 count:  9
152000 152999
해당가격대 숙소 count:  13
153000 153999
해당가격대 숙소 count:  8
154000 154999
해당가격대 숙소 count:  18
155000 155999
해당가격대 숙소 count:  17
156000 156999
해당가격대 숙소 count:  11
157000 157999
해당가격대 숙소 count:  12
158000 158999
해당가격대 숙소 count:  14
159000 159999
해당가격대 숙소 count:  53
160000 160999
해당가격대 숙소 count:  45
161000 161999
ERR: btw 161000 and 161999|'explore_tabs'
ERR: btw 161000 and 161999|'explore_tabs'
해당가격대 숙소 count:  47
162000 162999
해당가격대 숙소 count:  8
163000 163999
해당가격대 숙소 count:  7
164000 164999
ERR: btw 164000 and 164999|'explore_tabs'
ERR: btw 164000 and 164999|Expecting value: line 1 column 1 (char 0)
해당가격대 숙소 c

해당가격대 숙소 count:  17
401000 401999
402000 402999
해당가격대 숙소 count:  1
403000 403999
해당가격대 숙소 count:  1
404000 404999
405000 405999
406000 406999
407000 407999
408000 408999
해당가격대 숙소 count:  1
409000 409999
해당가격대 숙소 count:  1
410000 410999
해당가격대 숙소 count:  1
411000 411999
412000 412999
413000 413999
414000 414999
415000 415999
416000 416999
417000 417999
418000 418999
419000 419999
해당가격대 숙소 count:  1
420000 420999
해당가격대 숙소 count:  1
421000 421999
422000 422999
423000 423999
424000 424999
425000 425999
426000 426999
427000 427999
428000 428999
429000 429999
해당가격대 숙소 count:  2
430000 430999
해당가격대 숙소 count:  2
431000 431999
432000 432999
433000 433999
434000 434999
435000 435999
436000 436999
437000 437999
438000 438999
439000 439999
해당가격대 숙소 count:  3
440000 440999
해당가격대 숙소 count:  3
441000 441999
442000 442999
443000 443999
444000 444999
445000 445999
446000 446999
447000 447999
해당가격대 숙소 count:  1
448000 448999
해당가격대 숙소 count:  6
449000 449999
해당가격대 숙소 count:  18
450000 450999
해당가격대 숙소 coun

848000 848999
849000 849999
ERR: btw 849000 and 849999|'explore_tabs'
850000 850999
851000 851999
852000 852999
853000 853999
854000 854999
855000 855999
856000 856999
857000 857999
858000 858999
859000 859999
860000 860999
861000 861999
862000 862999
863000 863999
864000 864999
865000 865999
866000 866999
ERR: btw 866000 and 866999|'explore_tabs'
867000 867999
868000 868999
869000 869999
ERR: btw 869000 and 869999|'explore_tabs'
870000 870999
ERR: btw 870000 and 870999|'explore_tabs'
871000 871999
872000 872999
873000 873999
874000 874999
875000 875999
876000 876999
877000 877999
878000 878999
879000 879999
880000 880999
881000 881999
882000 882999
ERR: btw 882000 and 882999|'explore_tabs'
ERR: btw 882000 and 882999|Expecting value: line 1 column 1 (char 0)
883000 883999
ERR: btw 883000 and 883999|'explore_tabs'
884000 884999
885000 885999
886000 886999
ERR: btw 886000 and 886999|'explore_tabs'
해당가격대 숙소 count:  1
887000 887999
해당가격대 숙소 count:  1
888000 888999
해당가격대 숙소 count:  1
889000

해당 데이터 못가져옴
919000 919999
920000 920999
ERR: btw 920000 and 920999|Expecting value: line 1 column 1 (char 0)
ERR: btw 920000 and 920999|Expecting value: line 1 column 1 (char 0)
921000 921999
ERR: btw 921000 and 921999|Expecting value: line 1 column 1 (char 0)
ERR: btw 921000 and 921999|Expecting value: line 1 column 1 (char 0)
922000 922999
923000 923999
924000 924999
925000 925999
ERR: btw 925000 and 925999|Expecting value: line 1 column 1 (char 0)
926000 926999
927000 927999
ERR: btw 927000 and 927999|Expecting value: line 1 column 1 (char 0)
ERR: btw 927000 and 927999|Expecting value: line 1 column 1 (char 0)
ERR: btw 927000 and 927999|'explore_tabs'
해당 데이터 못가져옴
927000 927999
ERR: btw 927000 and 927999|Expecting value: line 1 column 1 (char 0)
ERR: btw 927000 and 927999|Expecting value: line 1 column 1 (char 0)
ERR: btw 927000 and 927999|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
927000 927999
928000 928999
ERR: btw 928000 and 928999|'explore_tabs'
ERR: btw 928000 and 92

ERR: btw 963000 and 963999|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
963000 963999
ERR: btw 963000 and 963999|Expecting value: line 1 column 1 (char 0)
ERR: btw 963000 and 963999|'explore_tabs'
ERR: btw 963000 and 963999|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
963000 963999
ERR: btw 963000 and 963999|Expecting value: line 1 column 1 (char 0)
964000 964999
ERR: btw 964000 and 964999|Expecting value: line 1 column 1 (char 0)
965000 965999
ERR: btw 965000 and 965999|'explore_tabs'
ERR: btw 965000 and 965999|Expecting value: line 1 column 1 (char 0)
ERR: btw 965000 and 965999|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
965000 965999
ERR: btw 965000 and 965999|Expecting value: line 1 column 1 (char 0)
ERR: btw 965000 and 965999|Expecting value: line 1 column 1 (char 0)
ERR: btw 965000 and 965999|Expecting value: line 1 column 1 (char 0)
해당 데이터 못가져옴
965000 965999
ERR: btw 965000 and 965999|Expecting value: line 1 column 1 (char 0)
966000 966999
ERR: btw 96

In [45]:
len(home.keys()) # 100원 단위 전체

14650

In [21]:
len(home.keys()) # 100원 단위 30000~33000

679